In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import glob

In [2]:
DATA_DIR = Path('../../results')
assert DATA_DIR.exists()

# use glob to get all files matching *.txt anywhere in the tree under DIRNAME
glob_string = str(DATA_DIR) + r'\**\*.txt'
filenames = glob.glob(glob_string, recursive=True)
filenames


['..\\..\\results\\jatos_results_data_20231009140618\\study_result_311537\\comp-result_436996\\data.txt',
 '..\\..\\results\\jatos_results_data_20231009140618\\study_result_311538\\comp-result_436997\\data.txt',
 '..\\..\\results\\jatos_results_data_20231009140618\\study_result_311538\\comp-result_437000\\data.txt',
 '..\\..\\results\\jatos_results_data_20231009140618\\study_result_311539\\comp-result_436998\\data.txt',
 '..\\..\\results\\jatos_results_data_20231009140618\\study_result_311540\\comp-result_436999\\data.txt']

In [3]:
import json

all_records = []
for fn in filenames:
    p = Path(fn)
    study = p.parts[-3]
    component = p.parts[-3]
    try:
        with open(fn, 'r', encoding='utf-8') as jf:
            # Load the JSON data and store it in the 'data' dictionary using the file path as the key
            js = json.load(jf)

            record_common = {}

            record_common['browser'] = js['context']['browser']['name']
            record_common['screen_w'] = js['context']['screen']['width']
            record_common['screen_h'] = js['context']['screen']['height']
            record_common['os'] = js['context']['system']['os']
            record_common['exp_start_date'] = js['context']['date']['startdateUTC']
            record_common['exp_start_time'] = js['context']['date']['starttime']

            extra_questions = js['data'][-1]
            record_common['favorite_team'] = extra_questions['favourite_team']
            record_common['favorite_color'] = extra_questions['favourite_color']
            # TODO: replace with questiona: 
            # * The usual outfit of any of your favourite teams include (blue/red/both/neither)
            # * Your favourite colours include (blue/red/both/neither)

            _records = []
            for i, datum in enumerate(js['data'][:-1]):  # -1 because the last entry seems to be empty  TODO: check this
                record = {}

                record['age'] = datum['age']
                record['response_side'] = datum['color_response']
                record['impaired_vision'] = datum['daltonic']  # TODO
                record['experience'] = datum['football']
                # datetime = datum['datetime']  # e.g. "Mon Oct 09 2023 15:45:58 GMT+0200 (Central European Summer Time)" or "Mon Oct 09 2023 15:46:03 GMT+0200 (Ora legale dell’Europa centrale)"
                record['gender'] = datum['gender']
                #is_browser = datum['is_browser']  # TODO: maybe needed if people do it on the computer
                #opensesame_codename = datum['opensesame_codename']
                #opensesame_version = datum['opensesame_version']
                record['session_id'] = datum['sessionid']
                record['filename'] = f"id{datum['image_nr']}_v{datum['version']}_r{datum['number_of_red']}_b{datum['number_of_blue']}.jpg"  # TODO
                record['trial_nr'] = i
                record['scene_id'] = datum['image_nr']  # TODO: check or infer from file name
                record['red_count'] = datum['number_of_red']
                record['blue_count'] = datum['number_of_blue']
                record['version'] = datum['version']
                record['expected_side'] = datum['correct_response']

                _records.append(record | record_common)

            
            all_records = all_records  + _records
    except Exception as e:
        print(f"Error loading JSON data from {fn}: {e}")
        continue

df = pd.DataFrame.from_records(all_records)
df.head()

Error loading JSON data from ..\..\results\jatos_results_data_20231009140618\study_result_311538\comp-result_436997\data.txt: Expecting value: line 4 column 1 (char 6489)
Error loading JSON data from ..\..\results\jatos_results_data_20231009140618\study_result_311540\comp-result_436999\data.txt: Expecting value: line 1 column 2 (char 1)


,age,response_side,impaired_vision,experience,gender,session_id,filename,trial_nr,scene_id,red_count,...,version,expected_side,browser,screen_w,screen_h,os,exp_start_date,exp_start_time,favorite_team,favorite_color
0,38,red,no,last-year,male,1696859204317,id3_v1_r6_b7.jpg,0,3,6,...,1,blue,Netscape,1920,1080,Win32,09-09-23,15:45:58,other,green
1,38,blue,no,last-year,male,1696859204317,id2_v1_r5_b6.jpg,1,2,5,...,1,blue,Netscape,1920,1080,Win32,09-09-23,15:45:58,other,green
2,38,red,no,last-year,male,1696859204317,id2_v2_r6_b5.jpg,2,2,6,...,2,red,Netscape,1920,1080,Win32,09-09-23,15:45:58,other,green
3,38,red,no,last-year,male,1696859204317,id1_v2_r7_b5.jpg,3,1,7,...,2,red,Netscape,1920,1080,Win32,09-09-23,15:45:58,other,green
4,38,red,no,last-year,male,1696859204317,id3_v2_r7_b6.jpg,4,3,7,...,2,red,Netscape,1920,1080,Win32,09-09-23,15:45:58,other,green
